### Collaborative Filtering avec Suprise

In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD

In [2]:
# Fonction de recommandation
def generate_recommendations(ratings_updated, favorite_movies):

    new_user_id = ratings_updated['userId'].max() + 1

    for i in favorite_movies:
        newdata = pd.DataFrame([[new_user_id, i, 5.0]], columns=['userId', 'tmdb_id', 'rating'])
        ratings_updated = pd.concat([ratings_updated, newdata], ignore_index=True)

    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(ratings_updated[['userId', 'tmdb_id', 'rating']], reader)
    svd = SVD()
    train_set = data.build_full_trainset()
    svd.fit(train_set)

    all_pred = []
    movie_id = []
    for movie in ratings_updated['tmdb_id'].unique().tolist():
        pred = svd.predict(new_user_id, movie)
        all_pred.append(pred.est)
        movie_id.append(pred.iid)

    result = pd.DataFrame(list(zip(movie_id, all_pred)), columns=['tmdb_id', 'predicted_rating'])
    result.sort_values(by='predicted_rating', ascending=False, inplace=True)
    
    return result.sort_values(by='predicted_rating', ascending=False).head(30)

In [3]:
# Importation du fichier
ratings_updated = pd.read_csv('src/Movielens_ratings_updated.csv')

# Liste de films favoris (exemple)
favorite_movies = [671, 672, 673]

# Test de la fonction de recommandation
generate_recommendations(ratings_updated, favorite_movies)

,tmdb_id,predicted_rating
416,2493,4.968172
6,278,4.742337
72,289,4.719342
26,1891,4.674760
66,567,4.669907
15,11,4.669447
23,77338,4.632882
388,22803,4.620422
392,8321,4.614303
24,497,4.601371
